<div style="text-align: right"> CS824 - Lab 4a (2022) </div>

# Simple application of Bayes Law

As usual, follow those parts of the lab that have been set up - sometimes involving aspects of Python that you may not yet be familiar with, but try to make sure you get a feel for what is happening in terms of **probability** and **Bayes**...  There are various 'exercises' which are the elements that should form part of this week's submission ("SUBMIT").


# Part 1: Using Bayes to understand medical diagnosis

Depending on how familiar you are with the terms from medical diagnosis you may need to refer to the formulae in mini-lecture 4.2 to translate the example below into a function that uses Bayes Law to properly interpret the results from diagnostic tests.

## Exercise 1a (SUBMIT)
Build a function in Python that takes in the following three parameters:
- the prevalence of some disease in a population (sometimes called the 'base rate')
- the sensitivity of a diagnostic test (also referred to as the 'true positive rate')
- the specificity of a diagnostic test (also referred to as the 'true negative rate')

<br> and returns a value that represents the probability that a patient from that population has the disease based on getting a positive result from such a test.

Start with the example from the mini-lecture, i.e.:
- prevalence of some rare disease = 1 in 5,000
- sensitivity = 99%
- specificity = 95%

<br>Given these parameters your function should return a value of around 0.4%, - i.e. the P(disease | +test).


In [56]:
def ex1a(prevalence,sensitivity,specificity):
    """returns a value that represents the probability that a patient from that population has the disease based on getting a positive result from such a test.
    P(D|+) = P(+|D)*P(D)/P(+)
    P(D) = prevalence
    P(+|D) = sensitivity
    P(+) = (prevalence*sensitivity + (1-specificity)*(1-prevalence))
    """
    return prevalence*sensitivity/(prevalence*sensitivity + (1-specificity)*(1-prevalence))

In [57]:
ex1a(1/5000,0.99,0.95)

0.003945166175181315

## Exercise 1b (SUBMIT)
How (if at all) does your belief that this patient has the disease alter if she gets a **second** positive test? (Assume this has the same 'test characteristics' but is run by a different lab, just so that we know the two tests are in some way independent.) Does that makes sense to you?


#### The result that I came up with increased the probability to just over 7%. This is now starting to get a bit more serious/worrying...  Did you get the same result? Do you trust this value?  If not, why not? 

In [58]:
ex1a(ex1a(1/5000,0.99,0.95),0.99,0.95)

0.07272066243248045

I trust it because it has increased in magnitude quite a bit from the last time; and the first test had a very low probability because the disease is very rare and the amount of false positives is big enough to not make the next value as big (i.e. the specificity is not as big as it can be).

## Exercise 1c (SUBMIT)
What about if the level of specificity is equal to that of the sensitivity - i.e. both are at 99%.  How does this alter things? (Look at this for both the first and subsequent runs of this 'new improved' test.) 

#### Did the new outcomes surprise you?  Again would you intuitively accept/'believe' these numbers? How have these exercises impacted on your understanding of how 'sensitivity' and 'specificity' in diagnositc tests should be understood?

In [59]:
ex1a(1/5000,0.99,0.99)

0.019419380149078055

In [60]:
ex1a(ex1a(1/5000,0.99,0.99),0.99,0.99)

0.6622297297297294

As it was said in the previous exercise, the specificity marked how probable it was to get a false positive. In this case, the probability is much lower, so it is more probable that our positives are real. In this case, it increases to 2% (still pondering the rarity of the disease) and 66% for the second test.



## Exercise 1d (SUBMIT)
Have a look at the situation for **COVID-19**:
- you will need to check the Web for a suitable base rate to explore this issue, as well as commonly accepted test characteristics. If you find suggestions that specificity is 100% (as some do) then you may be more interested in the presence of COVID in an indivdual who has tested NEGATIVE (explain why the positive outome case is 'uninteresting' in this context). You may wish to look at both PCR and lateral flow tests (LFTs) when carrying out this exercise.

<br> Which of the parameters seems to have the largest impact on the outcome (i.e. your belief that there really is something that you should be worried about and/or relaxed enough to ignore, given different outcomes)?


In [61]:
prevalence = 0.02 # https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19latestinsights/infections
pcr_sensitivity = 0.87
pcr_specificity = 0.99 # https://www.infectiousdiseaseadvisor.com/home/topics/covid19/covid-19-diagnosis-pcr-testing-value/
lft_sensitivity = 0.76
lft_specificity = 0.99 # https://www.ox.ac.uk/news/2020-11-11-oxford-university-and-phe-confirm-lateral-flow-tests-show-high-specificity-and-are

In [62]:
ex1a(prevalence,pcr_sensitivity,pcr_specificity)

0.6397058823529409

In [63]:
ex1a(prevalence,lft_sensitivity,lft_specificity)

0.6079999999999998

If the specificity is 100% then there exists no false positives, so if you are positive, the probability of you having the disease is 100%. This is why it would be more interesting to study the case in which you are negative.

It can be seen that the COVID tests have all very high specificity. This means that people that test positive can be certain they are quarantining for a real reason. However, it seems like having a higher sensitivity would be advisable in order to reduce the false negatives and reduce the disease spread.

So if you test positive, you are quite sure that you have the disease. However, if you test negative, you might want to take a second test just in case.

# Part 2: M&Ms and Bayes

So-called 'urn' (or 'bag') problems don't always present in the *classic* manner we came across last week - dating back to Bernoulli. The problem below is in the same basic format as the 'urn' problems of last week but allow for a little bit of a twist. This example is based on an example around the American candy, M&Ms and their production over time, and is based on an entry from [Allen Downey's](https://www.allendowney.com/blog/) excellent blog (though as always, any errors introduced in editing are entirely my fault!)

> The blue M&M was introduced in 1995.  Before then, the colour mix in a bag of M&Ms was (30% Brown, 20% Yellow, 20% Red, 10% Green, 10% Orange, 10% Tan).  After 1995 it was (24% Blue, 20% Green, 16% Orange, 14% Yellow, 13% Red, 13% Brown).

> A friend of mine has two bags of M&Ms, and he tells me that one is from 1994 and one from 1996 but won't tell me which is which. He gives me a yellow M&M, what is the probability that this came from the 1994 bag?

Now that you have picked up on the basic ideas of Bayesian reasoning you should see that this seems to be a good candidate, in that we are asked to estimate the probability of a hypothesis (that the yellow M&M is from 1994) given some evidence. It is not immediately obvivous how we might do this but we should be able to take the probabilities that we have been given and work in the 'opposite' direction, i.e. what is the probability that we would have recieved this 'evidence' under the different hypotheses.


You may feel that all this reasoning is 'over-kill', but we are going to add a bit more evidence in a moment, so breaking down our thinking in this way will ultimately pay off...

Before we see the colour of the M&M, there are two hypotheses, `A` and `B`, both with equal probability:

    A: the M&M was from the 94 bag
    B: the M&M was from the 96 bag
    P(A) = P(B) = 0.5
    
Then we get some evidence:
    
    E: the M&M is yellow
    
We want to know the probability of hypothesis `A`, given the evidence:
    
    P(A | E)
    
We *could* enumerate the sample space to estimate this but that will start to become more tricky as we are given more evidence (see below). So instead let's take what we know from Bayes' Theorem, namely that:
    
    P(A | E) = P(E | A) * P(A) / P(E)
    
The quantities on the right-hand-side of this equation are all pretty straight-forward to calculate:
    
    P(E | A) = 0.20
    P(E | B) = 0.14
    P(A)     = 0.5
    P(B)     = 0.5
    P(E)     = P(E | A) * P(A) + P(E | B) * P(B) 
             = 0.20     * 0.5  + 0.14     * 0.5   =   0.17
    
We can use Bayes Law to get a final answer:
   
    P(A | E) = P(E | A) * P(A) / P(E) 
             = 0.20     * 0.5  / 0.17 
             = 0.588

i.e. There is about a 59% chance that this yellow M&M was taken was the 1994 bag.  

You may say, "well that was a LOT of work to calcuate what was a lot more obvious, by simply noting that 20/34 (i.e. 1994_yellows / all_yellows) is the proportion / likelihood of the yellow coming from the 1994 bag". And you would be correct, but see whether your mind might change when faced with the next question...  


## Exercise 2a (SUBMIT)

Let's think about a slightly more 'interesting' situation...  Say this time that our friend draws one M&M from each bag - the first one is yellow and the seceond one is green. Now how might you go about estimating the probability that the yellow M&M came from the 1994 bag?

<br>
You should be able to use the approach outlined above to estimate this value. It would be great to build a little 'Bayes estimator' function in Python to explore this and then other selections of M&Ms, so try to create that rather than (or in addition to) working our each option *by hand* (as we did above).

<br>
(I estimated that there was now a 74% chance that the yellow M&M came from the 1994 bag... Did you find the same?  Does this value make sense to you?)


In [64]:
mybag94 = {"brown":0.3,"yellow":0.2,"red":0.2,"green":0.1,"orange":0.1,"tan":0.1,"blue":0}
mybag96 = {"brown":0.13,"yellow":0.14,"red":0.13,"green":0.2,"orange":0.16,"tan":0,"blue":0.24}

In [65]:
# BAG1 is 94, given BAG1 extraction is yellow and uniform prior
first_extraction = ex1a(0.5, mybag94['yellow'],1-mybag96['yellow'])
first_extraction

0.5882352941176471

In [66]:
# BAG1 is 94, given BAG2 extraction is green and first_extraction prior
second_extraction = ex1a(first_extraction,mybag96['green'],1-mybag94['green'])
second_extraction

0.7407407407407408

The value makes sense. It is more probable that BAG1 is the 94 because there are more yellows than in 96, and there are more greens in the 96 than in the 94. So both higher probabilities add up to a high number.

# Do we really need Bayes Law?

An alternative approach was suggested by Peter Norvig as he reflected on Allen Downey's example...

In a sense it is curious that this problem comes from a section titled by Downey as **My favorite Bayes's Theorem Problems**, as you might expect that you would need to invoke Bayes Theorem to solve it. However, the enumerative approach outlined below shows that is not strictly necessary.

Yes, Bayes Theorem allows you to do less calculation but at the cost of more algebra; that is a great trade-off if you are working with pencil and paper. Enumerating the state space allows you to do less algebra at the cost of more calculation; sometimes a good trade-off if you have a computer. Using both approaches may help you to more fully understand Bayes theorem and how it works (or at least confirm that the algebra agrees with the enumerated outcome!).

We can use some of the code that was introduced to us in Norvig's tutorial examples of urn problems from last week. We will start by re-introducing the various functions that he developed there, though here they are replaced with slightly 'enhanced' versions (but essentially acheive the same results).


In [67]:
from fractions import Fraction

is_predicate = callable

def P(event, space): 
    """The probability of an event, given a sample space of equiprobable outcomes. 
    event: a collection of outcomes, or a predicate that is true of outcomes in the event. 
    space: a set of outcomes or a probability distribution of {outcome: frequency} pairs."""
    if is_predicate(event):
        event = such_that(event, space)
    if isinstance(space, ProbDist):
        return sum(space[o] for o in space if o in event)
    else:
        return Fraction(len(event & space), len(space))
    
def such_that(predicate, space): 
    """The outcomes in the sample pace for which the predicate is true.
    If space is a set, return a subset {outcome,...};
    if space is a ProbDist, return a ProbDist {outcome: frequency,...};
    in both cases only with outcomes where predicate(element) is true."""
    if isinstance(space, ProbDist):
        return ProbDist({o:space[o] for o in space if predicate(o)})
    else:
        return {o for o in space if predicate(o)}
    

To tackle this particulars of this problem, we'll need to represent the probability distributions of the difference M&Ms in each bag.  We define a `ProbDist` function and then use it to create the distribution of differing colours of M&Ms contained in `bag94` and `bag96`.

In [68]:
class ProbDist(dict):
    "A Probability Distribution; an {outcome: probability} mapping."
    def __init__(self, mapping=(), **kwargs):
        self.update(mapping, **kwargs)
        # Make probabilities sum to 1.0; assert no negative probabilities
        total = sum(self.values())
        for outcome in self:
            self[outcome] = self[outcome] / total
            assert self[outcome] >= 0

In [69]:
bag94 = ProbDist(brown=30, yellow=20, red=20, green=10, orange=10, tan=10)
bag96 = ProbDist(blue=24, green=20, orange=16, yellow=14, red=13, brown=13)

In [70]:
# You can check the contents of a bag (or more correctly the propotions of differing colour contents in a bag)
bag94

{'brown': 0.3,
 'yellow': 0.2,
 'red': 0.2,
 'green': 0.1,
 'orange': 0.1,
 'tan': 0.1}

We can now define `two_MM` as the joint distribution - the sample space for picking two M&Ms, one from each of the bags defined by these two distributions. The outcome `'brown - blue'` means that a brown M&M was selected from the 1994 bag and that a blue one came from the 1996 bag. There will be no entry for `blue - brown` as there were no blue M&Ms in the bags from before 1995, but for many selections we are able to find the values for various `a - b` and `b - a` outcomes.

In [71]:
def joint(A, B, sep=''):
    """The joint distribution of two independent probability distributions. 
    Result is all entries of the form {a+sep+b: P(a)*P(b)}"""
    return ProbDist({a + sep + b: A[a] * B[b]
                    for a in A
                    for b in B})

two_MM = joint(bag94, bag96, ' - ')
two_MM

{'brown - blue': 0.07199999999999997,
 'brown - green': 0.05999999999999997,
 'brown - orange': 0.04799999999999998,
 'brown - yellow': 0.04199999999999998,
 'brown - red': 0.038999999999999986,
 'brown - brown': 0.038999999999999986,
 'yellow - blue': 0.04799999999999998,
 'yellow - green': 0.03999999999999999,
 'yellow - orange': 0.03199999999999999,
 'yellow - yellow': 0.02799999999999999,
 'yellow - red': 0.025999999999999992,
 'yellow - brown': 0.025999999999999992,
 'red - blue': 0.04799999999999998,
 'red - green': 0.03999999999999999,
 'red - orange': 0.03199999999999999,
 'red - yellow': 0.02799999999999999,
 'red - red': 0.025999999999999992,
 'red - brown': 0.025999999999999992,
 'green - blue': 0.02399999999999999,
 'green - green': 0.019999999999999993,
 'green - orange': 0.015999999999999993,
 'green - yellow': 0.013999999999999995,
 'green - red': 0.012999999999999996,
 'green - brown': 0.012999999999999996,
 'orange - blue': 0.02399999999999999,
 'orange - green': 0.019

So now we can look at the possible ways in which we could end up with a "one is yellow and one is green" outcome.

In [72]:
def yellow_and_green(outcome): return 'yellow' in outcome and 'green' in outcome

such_that(yellow_and_green, two_MM)

{'yellow - green': 0.7407407407407408, 'green - yellow': 0.25925925925925924}

As you can see, these two numbers must sum to 1... if we have a 'yellow and green' outcome, then either we picked them in the order of the 94/96, or order of the 96/94, bags.

We therefore don't really need this last/next cell, but if we wanted to test in Python...

We can answer the question: given that we got a yellow and a green (but don't know which comes from which bag), what is the probability that the yellow came from the 1994 bag?

In [73]:
def yellow94(outcome): return outcome.startswith('yellow')

P(yellow94, such_that(yellow_and_green, two_MM))

0.7407407407407408

So, we end up with the same outcome as when we used Bayes' Rule in the previous section. Take some time (in the little exercise below) to reflect on these two approaches with some additional examples and provide some comments on your thoughts regarding the approach/outcomes.


## Exercise 2b (SUBMIT)

Try some alternative outcomes - say that you were given an **orange and a red M&M**.  What is the probability that the orange M&M came from the 1994 bag?  

What about if you got a **tan and a red**?  What is the probabilty that the red came from the 1994 bag?

You should try both the 'Bayesian' and the enumerative (Norvig's) approach.  Which did you find easier?  Which seemed more intuitive to you and why?

<br>

In [74]:
def orange_and_red(outcome): return 'orange' in outcome and 'red' in outcome
def orange94(outcome): return outcome.startswith('orange')

P(orange94, such_that(orange_and_red, two_MM))

0.2888888888888889

In [75]:
# BAG1 is 94, given BAG1 extraction is orange and uniform prior
ex2b_1 = ex1a(0.5, mybag94['orange'],1-mybag96['orange'])
# BAG1 is 94, given BAG2 extraction is red and extraction 1
ex2b_2 = ex1a(ex2b_1, mybag96['red'],1-mybag94['red'])
ex2b_2

0.2888888888888889

In [76]:
def tan_and_red(outcome): return 'tan' in outcome and 'red' in outcome
def red94(outcome): return outcome.startswith('red')

P(red94, such_that(tan_and_red, two_MM))

0

In [77]:
# BAG1 is 94, given BAG2 extraction is tan and uniform prior ->>> this is 0!!! this is why it cannot happen
ex2b2_1 = ex1a(0.5, mybag96['tan'],1-mybag94['tan'])
print(ex2b2_1)
# BAG1 is 94, given BAG1 extraction is red and extraction 1
ex2b2_2 = ex1a(ex2b2_1, mybag94['red'],1-mybag96['red'])
ex2b2_2

0.0


0.0

I personally find the Bayesian approach more intuitive because of the updating of beliefs system, which is methodical. The counting approach feels more like a black box. However it is also easier to code and to generalise.